# Construct Datasets for benchmarking tasks

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-10-08  

## Dataset1. Enzyme None-enzyme Dataset
The enzyme dataset is consists of two parts: a training set and a testing set. The training set is from snapshot Feb-2018 and excludes those deleted items in snapshot Jun-2020. The training set is consists of 467,973 records, of which 222,290 are enzymes, and 245,683 are not enzymes. The testing set is from snapshot Jun-2020 and excludes these items that appeared in snapshot Feb-2018. The testing set is consists of 8033 records, of which 3579 are enzymes, and 4454 are not-enzymes. Unlike previous works,  we did not filter any sequences in terms of length and homology to make the data more inclusive. We make a label for each sequence, 1 for enzyme and 0 for not-enzyme. 

## Dataset2. Enzyme Quantity Dataset
The enzyme quantity dataset only contains enzyme data, contain 13,108 records. The function quantity ranges from 2 to 10.

## Dataset 3: EC number Dataset

Similar to the enzyme quantity dataset, the EC number dataset is consists of 225,221 enzyme records, 221,642 are training-set, and the rest 3579 are testing-set, covering 4852 EC numbers. Up to Feb 2020, there still exist 267 EC numbers that the model can not handle in the benchmarking. Thus, we exclude the sequences with these 267 EC numbers in the evaluation process. But, this problem can be resolved in the production scenario because we use the entire data from Swiss-Prot. Now the EC coverage is 5307 and can be automatically extended, for the training is real-time based on the publication of Swiss-Prot every 8 weeks. 

## 1. Import packages

In [1]:
import numpy as np
import pandas as pd

import os
import re
import time
import datetime
import sys
from tqdm import tqdm
import config as cfg
from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import exact_ec_from_uniprot as exactec
import minitools as mtool
import embedding_esm as esmebd
import embedding_unirep as unirep

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize() # 初始化该这个b...并行库


%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Define Functions

In [3]:
# install axel for download dataset
def install_axel():
    isExists = !which axel
    if 'axel' in str(isExists[0]):
        return True
    else:
        !sudo apt install axel -y

# add missing '-' for ec number
def refill_ec(ec):   
    if ec == '-':
        return ec
    levelArray = ec.split('.')
    if  levelArray[3]=='':
        levelArray[3] ='-'
    ec = '.'.join(levelArray)
    return ec

def specific_ecs(ecstr):
    if '-' not in ecstr or len(ecstr)<4:
        return ecstr
    ecs = ecstr.split(',')
    if len(ecs)==1:
        return ecstr
    
    reslist=[]
    
    for ec in ecs:
        recs = ecs.copy()
        recs.remove(ec)
        ecarray = np.array([x.split('.') for x in recs])
        
        if '-' not in ec:
            reslist +=[ec]
            continue
        linearray= ec.split('.')
        if linearray[1] == '-':
            #l1 in l1s and l2 not empty
            if (linearray[0] in  ecarray[:,0]) and (len(set(ecarray[:,0]) - set({'-'}))>0):
                continue
        if linearray[2] == '-':
            # l1, l2 in l1s l2s, l3 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (len(set(ecarray[:,2]) - set({'-'}))>0):
                continue
        if linearray[3] == '-':
            # l1, l2, l3 in l1s l2s l3s, l4 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (linearray[2] in  ecarray[:,2]) and (len(set(ecarray[:,3]) - set({'-'}))>0):
                continue
                
        reslist +=[ec]
    return ','.join(reslist)

#format ec
def format_ec(ecstr):
    ecArray= ecstr.split(',')
    ecArray=[x.strip() for x in ecArray] #strip blank
    ecArray=[refill_ec(x) for x in ecArray] #format ec to full
    ecArray = list(set(ecArray)) # remove duplicates
    
    return ','.join(ecArray)

## 3. Download rawdata from unisprot

In [6]:
# download location ./tmp
install_axel()
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz -o ./data/uniprot_sprot_latest.dat.gz -q -c

## 4. Extract records from rawdata

In [5]:
# 2018 data
! tar -zxvf ./tmp/uniprot_sprot-only2018_02.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2018.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

# 2020 data
! tar -zxvf ./tmp/uniprot_sprot-only2020_06.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2020.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

uniprot_sprot.dat.gz
uniprot_sprot.fasta.gz
uniprot_sprot_varsplic.fasta.gz
uniprot_sprot.xml.gz
uniprot_sprot.dat.gz
uniprot_sprot.fasta.gz
uniprot_sprot_varsplic.fasta.gz
uniprot_sprot.xml.gz


In [7]:
exactec.run_exact_task(infile=cfg.DATADIR+'uniprot_sprot_latest.dat.gz', outfile=cfg.DATADIR+'sprot_latest.tsv')

565254it [04:38, 2032.46it/s]

finished use time 277.883 s


## 5. Load records

In [10]:
#加载数据并转换时间格式
sprot_latest = pd.read_csv(cfg.DATADIR+'sprot_latest.tsv', sep='\t',header=0) #读入文件
sprot_latest = mtool.convert_DF_dateTime(inputdf = sprot_latest)

## 6. Preprocessing
### 6.1 Drop Duplicates

In [12]:
sprot_latest.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot_latest.reset_index(drop=True, inplace=True)


### 6.2 format EC

In [13]:
#sprot_latest
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: format_ec(x))
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: specific_ecs(x))
sprot_latest['functionCounts'] = sprot_latest.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

In [14]:
sprot_latest

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P0A6V3,GLGC_ECO57,True,False,1,2.7.7.27,4,1986-07-21,2007-01-23,2021-06-02,MVSLEKNDHLMLARQLPLKSVALILAGGRGTRLKDLTNKRAKPAVH...,431
1,P0AEE5,DGAL_ECOLI,False,False,0,-,0,1986-07-21,1987-08-13,2021-06-02,MNKKVLTLSAVMASMLFGAAAHAADTRIGVTIYKYDDNFMSVVRKA...,332
2,P01508,CECB_HYACE,False,False,0,-,0,1986-07-21,1987-08-13,2021-06-02,MNFSRIFFFVFALVLALSTVSAAPEPKWKVFKKIEKMGRNIRNGIV...,62
3,P01936,HBA_EULFU,False,False,0,-,0,1986-07-21,1986-07-21,2021-04-07,VLSPADKTNVKTAWNAVGGQAGEHGAEALERMFLSFPTTKTYFPHF...,141
4,P01949,HBA_ERIEU,False,False,0,-,0,1986-07-21,1986-07-21,2021-04-07,VLSATDKANVKTFWGKLGGHGGEYGGEALDRMFQAHPTTKTYFPHF...,141
...,...,...,...,...,...,...,...,...,...,...,...,...
477257,C0HLS6,IBBA4_HYAOR,False,False,0,-,0,2021-06-02,2021-06-02,2021-06-02,WPPVEGRPSCKRCGACTRMWPPEANRCVCDDIVPQCHEGCSKCEKV...,67
477258,A0A0D1BUH6,ATR1_USTMA,False,False,0,-,0,2021-06-02,2015-04-29,2021-06-02,MRFRSDSRADHQHPKKQGSMDPDTIQALKYQDRSSSSSSNNKPKEK...,1382
477259,D0LHE5,BMCP_HALO1,False,False,0,-,0,2021-06-02,2009-12-15,2021-06-02,MVLGKVVGTVVASRKEPRIEGLSLLLVRACDPDGTPTGGAVVCADA...,96
477260,P0DUN4,PA2A2_BOTBZ,True,False,1,3.1.1.4,4,2021-06-02,2021-06-02,2021-06-02,NLWQFEMLIMKIAKTSGFMFYSSYGCYCGWGGHGRPQDASDRCCFV...,92


### 6.3 Get Tainset

In [20]:
train = sprot_latest.iloc[:,np.r_[0,2:7,10:12]]

### 6.4 Trim string

In [21]:
with pd.option_context('mode.chained_assignment', None):
    train.ec_number = train.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    train.seq = train.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

### 6.5 Save train latest

In [22]:
train.to_feather(cfg.DATADIR + 'sprot_latest.feature')

### 6.6 ESM embedding 

In [4]:
train= pd.read_feather(cfg.DATADIR + 'sprot_latest.feature')

In [4]:
# !pip install fair-esm
tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=train, model='esm1b_t33_650M_UR50S',seqthres=1022)
tr_rep0.to_feather(cfg.DATADIR + 'sprot_latest_rep0.feather')
tr_rep32.to_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
tr_rep33.to_feather(cfg.DATADIR + 'sprot_latest_rep33.feather')

Transferred model to GPU


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 477262/477262 [6:21:00<00:00, 20.88it/s]


### 6.7 Unirep embedding

In [ ]:
tr_unirep = unirep.getunirep(train, 200)
tr_unirep.to_feather(cfg.DATADIR + 'sprot_latest_unirep.feather')

 73%|███████████████████████████████████████████████████████████████████████████████████████████████▉                                    | 1734/2387 [47:50:42<21:30:00, 118.53s/it]

## 7. Build benchmarking datasets
### 7.1 Task 1 isEnzyme

In [15]:
task1_train = train.iloc[:,np.r_[0,7,1]]
task1_test = test.iloc[:,np.r_[0,7,1]]

task1_train.to_feather(cfg.DATADIR + 'task1/train.feather')
task1_test.to_feather(cfg.DATADIR + 'task1/test.feather')

### 7.2 Task2 Function Counts

In [16]:
task2_train = train[train.functionCounts >0]
task2_train.reset_index(drop=True, inplace=True)
task2_train = task2_train.iloc[:,np.r_[0,7,3]]

task2_test = test[test.functionCounts >0]
task2_test.reset_index(drop=True, inplace=True)
task2_test = task2_test.iloc[:,np.r_[0,7,3]]

task2_train.to_feather(cfg.DATADIR + 'task2/train.feather')
task2_test.to_feather(cfg.DATADIR + 'task2/test.feather')


### 7.3 Task3 EC Number

In [13]:
task3_train = train[train.functionCounts >0]
task3_train.reset_index(drop=True, inplace=True)
task3_train = task3_train.iloc[:,np.r_[0,7,4]]

task3_test = test[test.functionCounts >0]
task3_test.reset_index(drop=True, inplace=True)
task3_test = task3_test.iloc[:,np.r_[0,7,4]]

task3_train.to_feather(cfg.DATADIR + 'task3/train.feather')
task3_test.to_feather(cfg.DATADIR + 'task3/test.feather')
